# Limpieza, transformación y valoración de cálculo de indicadores SEPIID

 Análisis de datos exploratorio de los registros administrativos de: 
- Uso de salas de lactancia julio-dic 2024
- Seguro de gastos médicos mayores 

### Objectivos
Transformar las variables claves
Estadísticas descriptivas
Aglomerar las columnas que se requieren para el cálculo de los siguientes indicadores:

1. Porcentaje de mujeres y de hombres que tienen acceso a prestaciones laborales e institucionales relacionadas con la conciliación corresponsable (acceso a trámites) 
2. Tiempo destinado al uso de las salas de lactancia



https://docs.google.com/spreadsheets/d/17GZsKvYDVkK2Wo5ydhHyoCs4bDcrg-VInfmoyuxEaPI/edit?gid=404889619#gid=404889619

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

## Uso de salas de lactancias durante el segundo semestre 2024

In [2]:
salas = pd.read_csv('/Users/karlacruz/Desktop/MIDI/BD/SalasLactancia.csv')

In [5]:
salas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1064 entries, 0 to 1063
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         1064 non-null   int64  
 1   N°                                 1063 non-null   float64
 2   Mes                                1064 non-null   object 
 3   Entidad federativa                 1064 non-null   object 
 4   Nombre del edificio                1064 non-null   object 
 5   Ubicacion de la sala de lactancia  1064 non-null   object 
 6   Nombre Completo de la usuaria      1064 non-null   object 
 7   Cargo                              1064 non-null   object 
 8   Adscripción/Procedencia            1064 non-null   object 
 9   Edad de la persona                 1064 non-null   object 
 10  Genero                             1064 non-null   object 
 11  Edad en meses  del/a bebé          1064 non-null   objec

In [7]:
for col in salas.select_dtypes(include=['object']).columns:
    print(f"--- {col} ---")
    print(salas[col].value_counts(dropna=False))
    print("\n")

--- Mes ---
Mes
NOVIEMBRE     239
DICIEMBRE     200
JULIO         189
AGOSTO        184
OCTUBRE       145
SEPTIEMBRE    107
Name: count, dtype: int64


--- Entidad federativa ---
Entidad federativa
CIUDAD DE MEXICO       337
ESTADO DE MEXICO       186
OAXACA                 149
SINALOA                112
DURANGO                 65
CHIHUAHUA               34
VERACRUZ                29
YUCATAN                 22
MICHOACAN               20
COAHUILA                19
NAYARIT                 19
JALISCO                 16
PUEBLA                  13
QUINTANA ROO            11
CAMPECHE                10
BAJA CALIFORNIA SUR      9
TAMAULIPAS               6
BAJA CALIFORNIA          3
SONORA                   2
MORELOS                  1
SAN LUIS POTOSI          1
Name: count, dtype: int64


--- Nombre del edificio ---
Nombre del edificio
EDIFICIO SEDE, SAN BARTOLO COYOTEPEC OAX                                                                                                                       

In [9]:
#convierte de obj to str la columna Nombre, y se cambia de nombre usuarias 
salas['usuarias'] = salas['Nombre Completo de la usuaria'].fillna('').astype(str).str.strip()

In [11]:
#Crear un IDunico para las usuarias
#002 responde al segundo semestre del año
salas['id_usuarias'] = '002' + salas['usuarias'] + '2024'

In [13]:
#Verifica
salas[[ 'usuarias', 'Nombre Completo de la usuaria', 'id_usuarias']].head(-1)

,usuarias,Nombre Completo de la usuaria,id_usuarias
0,AKCO,AKCO,002AKCO2024
1,AKCO,AKCO,002AKCO2024
2,AKCO,AKCO,002AKCO2024
3,DGL,DGL,002DGL2024
4,AKCO,AKCO,002AKCO2024
...,...,...,...
1058,ALRUP,ALRUP,002ALRUP2024
1059,ALRUP,ALRUP,002ALRUP2024
1060,ALRUP,ALRUP,002ALRUP2024
1061,ALRUP,ALRUP,002ALRUP2024


In [15]:
#cambiar formato de bases
pd.to_datetime(salas['Fecha (DD/MM/AAAA)'], format='%d/%m/%Y', errors='coerce')

0      NaT
1      NaT
2      NaT
3      NaT
4      NaT
        ..
1059   NaT
1060   NaT
1061   NaT
1062   NaT
1063   NaT
Name: Fecha (DD/MM/AAAA), Length: 1064, dtype: datetime64[ns]

In [17]:
salas['Edad'] = pd.to_numeric(salas['Edad de la persona'], errors='coerce')
salas['Tiempo de estancia en sala'] = pd.to_timedelta(salas['Tiempo de estancia en sala'], errors='coerce')

In [19]:
salas['duracion'] = salas['Tiempo de estancia en sala'].dt.total_seconds() / 3600

In [21]:
salas['Año'] = 2024

In [23]:
# Definir los intervalos y etiquetas
bins = [20, 30, 40, 50, 100]
labels = ['21-30', '31-40', '41-50', '51+']
# Crear variable categórica
salas['grup_edad'] = pd.cut(salas['Edad'], bins=bins, labels=labels)

In [25]:
salas.describe()

,Unnamed: 0,N°,Tiempo de estancia en sala,Edad,duracion,Año
count,1064.000000,1063.000000,1049,1061.000000,1049.000000,1064.0
mean,531.500000,532.770461,0 days 00:28:22.135367016,32.564562,0.472815,2024.0
std,307.294647,307.312566,0 days 00:10:49.978285587,4.445565,0.180550,0.0
min,0.000000,1.000000,0 days 00:00:00,22.000000,0.000000,2024.0
25%,265.750000,267.500000,0 days 00:20:00,29.000000,0.333333,2024.0
50%,531.500000,533.000000,0 days 00:30:00,32.000000,0.500000,2024.0
75%,797.250000,798.500000,0 days 00:31:00,36.000000,0.516667,2024.0
max,1063.000000,1064.000000,0 days 02:16:00,45.000000,2.266667,2024.0


In [27]:
variables = [
'Año',
'Mes',
'Entidad federativa',
'Nombre del edificio',
'id_usuarias',
'Ubicacion de la sala de lactancia',
'Edad',
'grup_edad',
'Genero',
'duracion',
'Fecha (DD/MM/AAAA)'
]


In [29]:
df = salas[variables]

In [31]:
df.head()

,Año,Mes,Entidad federativa,Nombre del edificio,id_usuarias,Ubicacion de la sala de lactancia,Edad,grup_edad,Genero,duracion,Fecha (DD/MM/AAAA)
0,2024,JULIO,CIUDAD DE MEXICO,AJUSCO 170,002AKCO2024,"CARRETERA PICACHO AJUSCO 170, COLONIA JARDINES...",30.0,21-30,FEMENINO,0.266667,2024-07-02
1,2024,JULIO,CIUDAD DE MEXICO,AJUSCO 170,002AKCO2024,"CARRETERA PICACHO AJUSCO 170, COLONIA JARDINES...",30.0,21-30,FEMENINO,0.333333,2024-07-03
2,2024,JULIO,CIUDAD DE MEXICO,AJUSCO 170,002AKCO2024,"CARRETERA PICACHO AJUSCO 170, COLONIA JARDINES...",30.0,21-30,FEMENINO,0.333333,2024-07-04
3,2024,JULIO,CIUDAD DE MEXICO,AJUSCO 170,002DGL2024,"CARRETERA PICACHO AJUSCO 170, COLONIA JARDINES...",33.0,31-40,FEMENINO,0.333333,2024-07-04
4,2024,JULIO,CIUDAD DE MEXICO,AJUSCO 170,002AKCO2024,"CARRETERA PICACHO AJUSCO 170, COLONIA JARDINES...",30.0,21-30,FEMENINO,0.316667,2024-07-05


In [33]:

print(df.isnull().sum())

Año                                   0
Mes                                   0
Entidad federativa                    0
Nombre del edificio                   0
id_usuarias                           0
Ubicacion de la sala de lactancia     0
Edad                                  3
grup_edad                             3
Genero                                0
duracion                             15
Fecha (DD/MM/AAAA)                    0
dtype: int64


In [35]:
#se borran los valores nan que no tieneb tiempo de duración (variable de interés)
df = df.dropna()

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1049 entries, 0 to 1063
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   Año                                1049 non-null   int64   
 1   Mes                                1049 non-null   object  
 2   Entidad federativa                 1049 non-null   object  
 3   Nombre del edificio                1049 non-null   object  
 4   id_usuarias                        1049 non-null   object  
 5   Ubicacion de la sala de lactancia  1049 non-null   object  
 6   Edad                               1049 non-null   float64 
 7   grup_edad                          1049 non-null   category
 8   Genero                             1049 non-null   object  
 9   duracion                           1049 non-null   float64 
 10  Fecha (DD/MM/AAAA)                 1049 non-null   object  
dtypes: category(1), float64(2), int64(1), object(7)


In [39]:
for col in df.select_dtypes(include=['object', 'category', 'float64']).columns:
    print(f"--- {col} ---")
    print(df[col].value_counts(dropna=False))
    print("\n")

--- Mes ---
Mes
NOVIEMBRE     233
DICIEMBRE     196
JULIO         185
AGOSTO        183
OCTUBRE       145
SEPTIEMBRE    107
Name: count, dtype: int64


--- Entidad federativa ---
Entidad federativa
CIUDAD DE MEXICO       334
ESTADO DE MEXICO       186
OAXACA                 145
SINALOA                112
DURANGO                 65
CHIHUAHUA               34
VERACRUZ                29
YUCATAN                 22
MICHOACAN               20
NAYARIT                 19
COAHUILA                18
JALISCO                 16
PUEBLA                  13
QUINTANA ROO            11
BAJA CALIFORNIA SUR      9
CAMPECHE                 9
BAJA CALIFORNIA          3
SONORA                   2
SAN LUIS POTOSI          1
TAMAULIPAS               1
Name: count, dtype: int64


--- Nombre del edificio ---
Nombre del edificio
EDIFICIO SEDE, SAN BARTOLO COYOTEPEC OAX                                                                                                                                                  

In [41]:
df.groupby('Entidad federativa')['duracion'].agg(['count', 'mean', 'sum']).reset_index()

,Entidad federativa,count,mean,sum
0,BAJA CALIFORNIA,3,0.388889,1.166667
1,BAJA CALIFORNIA SUR,9,0.425926,3.833333
2,CAMPECHE,9,0.744444,6.700000
3,CHIHUAHUA,34,0.474510,16.133333
4,CIUDAD DE MEXICO,334,0.458832,153.250000
5,COAHUILA,18,0.430556,7.750000
6,DURANGO,65,0.457179,29.716667
7,ESTADO DE MEXICO,186,0.378047,70.316667
8,JALISCO,16,0.490625,7.850000
9,MICHOACAN,20,0.414167,8.283333


In [45]:
df.to_csv("/Users/karlacruz/Desktop/MIDI/BD/salas.csv", index=False)

# Seguro de Gastos Médicos 

In [86]:
sgmm = pd.read_csv('/Users/karlacruz/Desktop/MIDI/BD/SGMM_23_24.csv')

In [88]:
sgmm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 1337 non-null   int64 
 1   numero                     1337 non-null   int64 
 2   mes                        1337 non-null   object
 3   estado                     1337 non-null   object
 4   ciudad                     1337 non-null   object
 5   adscripcion                1337 non-null   object
 6   circuito                   1337 non-null   int64 
 7   nivel                      1337 non-null   object
 8   expediente                 1337 non-null   int64 
 9   sp_homb_m                  1337 non-null   int64 
 10  sp_hom_f                   1337 non-null   int64 
 11  sp_muj_m                   1337 non-null   int64 
 12  sp_muj_f                   1337 non-null   int64 
 13  total_infancias_afiliadas  1337 non-null   int64 
 14  total_ni

In [90]:
sgmm.head()

,Unnamed: 0,numero,mes,estado,ciudad,adscripcion,circuito,nivel,expediente,sp_homb_m,sp_hom_f,sp_muj_m,sp_muj_f,total_infancias_afiliadas,total_ninas_afiliadas,total_ninos_afiliados,anno
0,0,1,ENERO,AGUASCALIENTES,AGUASCALIENTES,"ADMINISTRACION REGIONAL EN AGUASCALIENTES, AGU...",30,27,233234,1,0,0,0,1,0,1,2023
1,1,2,ENERO,AGUASCALIENTES,AGUASCALIENTES,TRIBUNAL COLEGIADO DE APELACION 30MO. CIRCUITO...,30,21,206519,0,1,0,0,1,1,0,2023
2,2,3,ENERO,BAJA CALIFORNIA,ENSENADA,"D.G.S.P. SERVICIO MEDICO EN ENSENADA, BAJA CAL...",15,21,231979,0,2,0,0,2,2,0,2023
3,3,4,ENERO,BAJA CALIFORNIA,MEXICALI,DEFENSORIA PUBLICA FEDERAL ADSCRITA A LA AMPF ...,15,27,204401,0,1,0,0,1,1,0,2023
4,4,5,ENERO,BAJA CALIFORNIA,MEXICALI,"JUZGADO 5TO DE DISTRITO EN MEXICALI, BAJA CALI...",15,7,45049,0,1,0,0,1,1,0,2023


In [108]:
sgmm['sexo_sp'] = ' '
sgmm['sp'] = 1

In [110]:
#Generar una variable categorica de sexo 
sgmm.loc[(sgmm['sp_muj_m'] >= 1) | (sgmm['sp_muj_f'] >= 1), 'sexo_sp'] = 'MUJER'
sgmm.loc[(sgmm['sp_homb_m'] >= 1) | (sgmm['sp_hom_f'] >= 1), 'sexo_sp']= 'HOMBRE'


In [112]:
for col in sgmm.select_dtypes(include=['object', 'category', 'float64']).columns:
    print(f"--- {col} ---")
    print(sgmm[col].value_counts(dropna=False))
    print("\n")

--- mes ---
mes
ENERO         156
ABRIL         137
JUNIO         137
MARZO         131
AGOSTO        129
OCTUBRE       129
MAYO          126
FEBRERO       118
SEPTIEMBRE     90
NOVIEMBRE      90
JULIO          76
DICIEMBRE      18
Name: count, dtype: int64


--- estado ---
estado
CIUDAD DE MEXICO       292
JALISCO                110
MEXICO                  80
NUEVO LEON              64
SINALOA                 60
COAHUILA                60
PUEBLA                  56
BAJA CALIFORNIA         53
TAMAULIPAS              51
VERACRUZ                41
SAN LUIS POTOSI         38
SONORA                  37
YUCATAN                 34
CHIHUAHUA               32
GUANAJUATO              31
NAYARIT                 29
GUERRERO                26
TABASCO                 26
MORELOS                 25
AGUASCALIENTES          25
QUERETARO               23
MICHOACAN               22
QUINTANA ROO            17
DURANGO                 16
CHIAPAS                 15
ZACATECAS               13
OAXACA          

In [114]:
sgmm.head()

,Unnamed: 0,numero,mes,estado,ciudad,adscripcion,circuito,nivel,expediente,sp_homb_m,sp_hom_f,sp_muj_m,sp_muj_f,total_infancias_afiliadas,total_ninas_afiliadas,total_ninos_afiliados,anno,sexo_sp,sp
0,0,1,ENERO,AGUASCALIENTES,AGUASCALIENTES,"ADMINISTRACION REGIONAL EN AGUASCALIENTES, AGU...",30,27,233234,1,0,0,0,1,0,1,2023,HOMBRE,1
1,1,2,ENERO,AGUASCALIENTES,AGUASCALIENTES,TRIBUNAL COLEGIADO DE APELACION 30MO. CIRCUITO...,30,21,206519,0,1,0,0,1,1,0,2023,HOMBRE,1
2,2,3,ENERO,BAJA CALIFORNIA,ENSENADA,"D.G.S.P. SERVICIO MEDICO EN ENSENADA, BAJA CAL...",15,21,231979,0,2,0,0,2,2,0,2023,HOMBRE,1
3,3,4,ENERO,BAJA CALIFORNIA,MEXICALI,DEFENSORIA PUBLICA FEDERAL ADSCRITA A LA AMPF ...,15,27,204401,0,1,0,0,1,1,0,2023,HOMBRE,1
4,4,5,ENERO,BAJA CALIFORNIA,MEXICALI,"JUZGADO 5TO DE DISTRITO EN MEXICALI, BAJA CALI...",15,7,45049,0,1,0,0,1,1,0,2023,HOMBRE,1


In [118]:
sgmm.groupby('sexo_sp')['sp'].agg(['count', 'mean', 'sum']).reset_index()

,sexo_sp,count,mean,sum
0,HOMBRE,751,1.0,751
1,MUJER,586,1.0,586


In [130]:
sgmm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 1337 non-null   int64 
 1   numero                     1337 non-null   int64 
 2   mes                        1337 non-null   object
 3   estado                     1337 non-null   object
 4   ciudad                     1337 non-null   object
 5   adscripcion                1337 non-null   object
 6   circuito                   1337 non-null   int64 
 7   nivel                      1337 non-null   object
 8   expediente                 1337 non-null   int64 
 9   sp_homb_m                  1337 non-null   int64 
 10  sp_hom_f                   1337 non-null   int64 
 11  sp_muj_m                   1337 non-null   int64 
 12  sp_muj_f                   1337 non-null   int64 
 13  total_infancias_afiliadas  1337 non-null   int64 
 14  total_ni

In [132]:
var = [
'mes',
'estado',
'ciudad',
'adscripcion',
'nivel',
'circuito',
'total_infancias_afiliadas',
'total_ninas_afiliadas',
'total_ninos_afiliados',
'anno',
'sexo_sp',
'sp',
]
#Censo ofrece del total cuantos accedieron y aquí vemos solo de organos jurisdiccionales
#% de personal organismos jurisdiccionales que afilio a SGMM por sexo y entidad, cargo

In [134]:
df_SGMM = sgmm[var]
df_SGMM

,mes,estado,ciudad,adscripcion,nivel,circuito,total_infancias_afiliadas,total_ninas_afiliadas,total_ninos_afiliados,anno,sexo_sp,sp
0,ENERO,AGUASCALIENTES,AGUASCALIENTES,"ADMINISTRACION REGIONAL EN AGUASCALIENTES, AGU...",27,30,1,0,1,2023,HOMBRE,1
1,ENERO,AGUASCALIENTES,AGUASCALIENTES,TRIBUNAL COLEGIADO DE APELACION 30MO. CIRCUITO...,21,30,1,1,0,2023,HOMBRE,1
2,ENERO,BAJA CALIFORNIA,ENSENADA,"D.G.S.P. SERVICIO MEDICO EN ENSENADA, BAJA CAL...",21,15,2,2,0,2023,HOMBRE,1
3,ENERO,BAJA CALIFORNIA,MEXICALI,DEFENSORIA PUBLICA FEDERAL ADSCRITA A LA AMPF ...,27,15,1,1,0,2023,HOMBRE,1
4,ENERO,BAJA CALIFORNIA,MEXICALI,"JUZGADO 5TO DE DISTRITO EN MEXICALI, BAJA CALI...",7,15,1,1,0,2023,HOMBRE,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1332,NOVIEMBRE,TAMAULIPAS,CIUDAD VICTORIA,DELEGACION DEL IFDP TAMAULIPAS,24,19,1,1,0,2024,HOMBRE,1
1333,NOVIEMBRE,TAMAULIPAS,CIUDAD VICTORIA,"JUZGADO 14TO DE DISTRITO EN CD. VICTORIA, TAMA...",25,19,1,0,1,2024,MUJER,1
1334,NOVIEMBRE,VERACRUZ,VILLA ALDAMA,"D.G.S.P. SERVICIO MEDICO EN VILLA ALDAMA, VERA...",27,7,1,1,0,2024,HOMBRE,1
1335,NOVIEMBRE,YUCATAN,MERIDA,JUZGADO DE DISTRITO EN MATERIA MERCANTIL FEDER...,27,14,1,1,0,2024,MUJER,1


In [136]:
print(df_SGMM.isnull().sum())

mes                          0
estado                       0
ciudad                       0
adscripcion                  0
nivel                        0
circuito                     0
total_infancias_afiliadas    0
total_ninas_afiliadas        0
total_ninos_afiliados        0
anno                         0
sexo_sp                      0
sp                           0
dtype: int64


In [140]:
df_SGMM.to_csv("/Users/karlacruz/Desktop/MIDI/BD/sgmm.csv", index=False)